In [1]:
import hail as hl
hl.init()

Running on Apache Spark version 2.4.5
SparkUI available at http://hail-m.us-central1-f.c.cncd-cncd.internal:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.61-3c86d3ba497a
LOGGING: writing to /hail-20210206-1403-0.2.61-3c86d3ba497a.log


In [2]:
recode = {f"{i}":f"chr{i}" for i in (list(range(1, 23)) + ['X', 'Y'])}
mt1 = hl.import_vcf('gs://pgr_wes_30k/GRCh38/chr12_30k_grch38.vcf.bgz', reference_genome='GRCh38', contig_recoding=recode, force_bgz=True)
cprint('1')
participant_ids = ['RM1547','X3218','V3711','X3107','V2526','V2029'
                   ,'V2586','V3289','PROMIS191039349']
ds_result.rows().show()
print(ds_result.count())
ds_result1 = ds_result.filter_cols(hl.literal(participant_ids).contains(ds_result.s))
print(ds_result1.count())
ds_result1.rows().show()

print('2')
ds_result1 = hl.split_multi_hts(ds_result1, permit_shuffle=True)
ds_result1.rows().show()

print('3')
print('Annotate Heterozygous Alleles')
ds_result1 = ds_result1.annotate_rows(hets = hl.agg.filter(ds_result1.GT.is_het(),hl.agg.collect(ds_result1.s)))
ds_result1.count()
print('Annotate N Hets')
ds_result4 = ds_result1.annotate_rows(n_het = hl.agg.count_where(ds_result1.GT.is_het()))
ds_result4.count()
print('Annotate Homozygous Alleles')
ds_result3 = ds_result4.annotate_rows(homs = hl.agg.filter(ds_result4.GT.is_hom_var(),hl.agg.collect(ds_result4.s)))
ds_result3.count()
print('Annotating N Homs')
ds_result5 = ds_result3.annotate_rows(n_hom = hl.agg.count_where(ds_result3.GT.is_hom_var()))

ds_result5.rows().show()

data3 = ds_result5.annotate_rows(
    info=ds_result5.info.annotate(hets=ds_result5.hets,
                             n_het=ds_result5.n_het,
                             homs=ds_result5.homs,
                             n_hom=ds_result5.n_hom)
                                )

hl.export_vcf(data3, "gs://test1haseeb/GALNT9_30K.vcf")

1


2021-02-06 14:03:58 Hail: INFO: Coerced sorted dataset
2021-02-06 14:03:58 Hail: INFO: reading 1 of 148 data partitions


+-----------------+------------+------+----------+----------+---------+
| locus           | alleles    | rsid |     qual | filters  | info.AC |
+-----------------+------------+------+----------+----------+---------+
| locus<GRCh38>   | array<str> | str  |  float64 | set<str> |   int32 |
+-----------------+------------+------+----------+----------+---------+
| chr12:132197059 | ["C","T"]  | NA   | 2.14e+03 | {}       |       2 |
| chr12:132197060 | ["G","A"]  | NA   | 4.42e+03 | {}       |       1 |
| chr12:132197060 | ["G","C"]  | NA   | 4.42e+03 | {}       |       0 |
| chr12:132197064 | ["C","T"]  | NA   | 8.46e+02 | {}       |       1 |
| chr12:132197067 | ["C","T"]  | NA   | 2.86e+03 | {}       |       2 |
| chr12:132197075 | ["C","T"]  | NA   | 3.74e+03 | {}       |       1 |
| chr12:132197076 | ["G","A"]  | NA   | 9.25e+03 | {}       |       1 |
| chr12:132197088 | ["G","T"]  | NA   | 3.67e+02 | {}       |       1 |
| chr12:132197092 | ["C","T"]  | NA   | 5.43e+05 | {}       |       6 |
| chr12:132197093 | ["G","A"]  | NA   | 9.38e+03 | {}       |       1 |
+-----------------+------------+------+----------+----------+---------+

+----------+---------------+------------+--------------------+------------+
| info.aaf | info.callRate | info.hetRA | info.hg19_uid      | info.homAA |
+----------+---------------+------------+--------------------+------------+
|  float64 |       float64 |      int32 | str                |      int32 |
+----------+---------------+------------+--------------------+------------+
| 3.24e-05 |      1.00e+00 |          2 | "12:132681604:C:T" |          0 |
| 1.62e-05 |      1.00e+00 |          1 | "12:132681605:G:A" |          0 |
| 0.00e+00 |      1.00e+00 |          0 | "12:132681605:G:C" |          0 |
| 1.62e-05 |      1.00e+00 |          1 | "12:132681609:C:T" |          0 |
| 3.24e-05 |      1.00e+00 |          2 | "12:132681612:C:T" |          0 |
| 1.62e-05 |      1.00e+00 |          1 | "12:132681620:C:T" |          0 |
| 1.62e-05 |      1.00e+00 |          1 | "12:132681621:G:A" |          0 |
| 1.62e-05 |      1.00e+00 |          1 | "12:132681633:G:T" |          0 |
| 9.73e-05 |      1.00e+00 |          6 | "12:132681637:C:T" |          0 |
| 1.62e-05 |      1.00e+00 |          1 | "12:132681638:G:A" |          0 |
+----------+---------------+------------+--------------------+------------+

+------------+---------------+---------------+---------------+---------------+
| info.homRR | info.maxDepth | info.maxHetAB | info.maxHomAB | info.medDepth |
+------------+---------------+---------------+---------------+---------------+
|      int32 |         int32 |       float64 |       float64 |         int32 |
+------------+---------------+---------------+---------------+---------------+
|      30825 |           223 |            NA |            NA |            39 |
|      30832 |           223 |            NA |            NA |            39 |
|      30833 |           223 |            NA |            NA |            39 |
|      30830 |           223 |            NA |            NA |            40 |
|      30829 |           223 |            NA |            NA |            40 |
|      30831 |           223 |            NA |            NA |            40 |
|      30831 |           223 |            NA |            NA |            40 |
|      30832 |           223 |            NA |            NA |            40 |
|      30827 |           223 |      5.24e-01 |            NA |            40 |
|      30832 |           223 |            NA |            NA |            40 |
+------------+---------------+---------------+---------------+---------------+

+---------------+---------------+---------------+---------------+
| info.medHetAB | info.medHomAB | info.minDpeth | info.minHetAB |
+---------------+---------------+---------------+---------------+
|       float64 |       float64 |         int32 |       float64 |
+---------------+---------------+---------------+---------------+
|      4.30e-01 |            NA |  

2021-02-06 14:04:14 Hail: INFO: Coerced sorted dataset
2021-02-06 14:04:14 Hail: INFO: reading 1 of 148 data partitions


(1092, 30833)


...

2021-02-06 14:07:05 Hail: INFO: Coerced sorted dataset
2021-02-06 14:07:05 Hail: INFO: reading 1 of 148 data partitions
2021-02-06 14:07:14 Hail: INFO: Coerced sorted dataset
2021-02-06 14:07:14 Hail: INFO: reading 1 of 148 data partitions


Annotating N Homs


2021-02-06 14:07:28 Hail: INFO: Coerced sorted dataset
2021-02-06 14:07:28 Hail: INFO: reading 1 of 148 data partitions
2021-02-06 14:07:47 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-02-06 14:07:56 Hail: INFO: Coerced sorted dataset
2021-02-06 14:07:56 Hail: INFO: reading 1 of 148 data partitions


+-----------------+------------+------+----------+----------+---------+
| locus           | alleles    | rsid |     qual | filters  | info.AC |
+-----------------+------------+------+----------+----------+---------+
| locus<GRCh38>   | array<str> | str  |  float64 | set<str> |   int32 |
+-----------------+------------+------+----------+----------+---------+
| chr12:132197059 | ["C","T"]  | NA   | 2.14e+03 | {}       |       2 |
| chr12:132197060 | ["G","A"]  | NA   | 4.42e+03 | {}       |       1 |
| chr12:132197060 | ["G","C"]  | NA   | 4.42e+03 | {}       |       0 |
| chr12:132197064 | ["C","T"]  | NA   | 8.46e+02 | {}       |       1 |
| chr12:132197067 | ["C","T"]  | NA   | 2.86e+03 | {}       |       2 |
| chr12:132197075 | ["C","T"]  | NA   | 3.74e+03 | {}       |       1 |
| chr12:132197076 | ["G","A"]  | NA   | 9.25e+03 | {}       |       1 |
| chr12:132197088 | ["G","T"]  | NA   | 3.67e+02 | {}       |       1 |
| chr12:132197092 | ["C","T"]  | NA   | 5.43e+05 | {}       |       6 |
| chr12:132197093 | ["G","A"]  | NA   | 9.38e+03 | {}       |       1 |
+-----------------+------------+------+----------+----------+---------+

+----------+---------------+------------+--------------------+------------+
| info.aaf | info.callRate | info.hetRA | info.hg19_uid      | info.homAA |
+----------+---------------+------------+--------------------+------------+
|  float64 |       float64 |      int32 | str                |      int32 |
+----------+---------------+------------+--------------------+------------+
| 3.24e-05 |      1.00e+00 |          2 | "12:132681604:C:T" |          0 |
| 1.62e-05 |      1.00e+00 |          1 | "12:132681605:G:A" |          0 |
| 0.00e+00 |      1.00e+00 |          0 | "12:132681605:G:C" |          0 |
| 1.62e-05 |      1.00e+00 |          1 | "12:132681609:C:T" |          0 |
| 3.24e-05 |      1.00e+00 |          2 | "12:132681612:C:T" |          0 |
| 1.62e-05 |      1.00e+00 |          1 | "12:132681620:C:T" |          0 |
| 1.62e-05 |      1.00e+00 |          1 | "12:132681621:G:A" |          0 |
| 1.62e-05 |      1.00e+00 |          1 | "12:132681633:G:T" |          0 |
| 9.73e-05 |      1.00e+00 |          6 | "12:132681637:C:T" |          0 |
| 1.62e-05 |      1.00e+00 |          1 | "12:132681638:G:A" |          0 |
+----------+---------------+------------+--------------------+------------+

+------------+---------------+---------------+---------------+---------------+
| info.homRR | info.maxDepth | info.maxHetAB | info.maxHomAB | info.medDepth |
+------------+---------------+---------------+---------------+---------------+
|      int32 |         int32 |       float64 |       float64 |         int32 |
+------------+---------------+---------------+---------------+---------------+
|      30825 |           223 |            NA |            NA |            39 |
|      30832 |           223 |            NA |            NA |            39 |
|      30833 |           223 |            NA |            NA |            39 |
|      30830 |           223 |            NA |            NA |            40 |
|      30829 |           223 |            NA |            NA |            40 |
|      30831 |           223 |            NA |            NA |            40 |
|      30831 |           223 |            NA |            NA |            40 |
|      30832 |           223 |            NA |            NA |            40 |
|      30827 |           223 |      5.24e-01 |            NA |            40 |
|      30832 |           223 |            NA |            NA |            40 |
+------------+---------------+---------------+---------------+---------------+

+---------------+---------------+---------------+---------------+
| info.medHetAB | info.medHomAB | info.minDpeth | info.minHetAB |
+---------------+---------------+---------------+---------------+
|       float64 |       float64 |         int32 |       float64 |
+---------------+---------------+---------------+---------------+
|      4.30e-01 |            NA |  

2021-02-06 14:08:33 Hail: WARN: export_vcf: ignored the following fields:
    'a_index' (row)
    'was_split' (row)
    'hets' (row)
    'n_het' (row)
    'homs' (row)
    'n_hom' (row)
2021-02-06 14:09:29 Hail: INFO: Coerced sorted dataset
2021-02-06 14:09:29 Hail: INFO: reading 1 of 148 data partitions
2021-02-06 14:10:02 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-02-06 14:10:24 Hail: INFO: Coerced sorted dataset
2021-02-06 14:10:24 Hail: INFO: reading 1 of 148 data partitions
2021-02-06 14:11:33 Hail: INFO: merging 1 files totalling 611.0K...
2021-02-06 14:11:33 Hail: INFO: while writing:
    gs://test1haseeb/GALNT9_30K.vcf
  merge time: 412.185ms
